<a href="https://colab.research.google.com/github/Jozdien/reward-side-channels/blob/main/Reward_Side_Channels.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [7]:
!pip install gym
!apt-get install python-opengl -y
!apt install xvfb -y

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-opengl is already the newest version (3.1.0+dfsg-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
xvfb is already the newest version (2:1.19.6-1ubuntu4.11).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [8]:

# Special gym environment
!pip install gym[atari]

# For rendering environment, you can use pyvirtualdisplay.
!pip install pyvirtualdisplay
!pip install piglet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
#Install spinningup on CoLab
!git clone https://github.com/AmritaANair/spinningup-logchanges.git
!cd spinningup
#!pip install -e . # this will incur error: File "setup.py" not found. Directory cannot be installed in editable mode: /content
!pip install -e spinningup-logchanges

Cloning into 'spinningup-logchanges'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (223/223), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 223 (delta 25), reused 223 (delta 25), pack-reused 0
Receiving objects: 100% (223/223), 23.77 MiB | 22.03 MiB/s, done.
Resolving deltas: 100% (25/25), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/spinningup-logchanges
  Attempting uninstall: spinup
    Found existing installation: spinup 0.2.0
    Can't uninstall 'spinup'. No files were found to uninstall.
  Running setup.py develop for spinup


In [10]:

# To activate virtual display 
# need to run a script once for training an agent as follows
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()

# This code creates a virtual display to draw game images on. 
# If you are running locally, just ignore it
import os
if type(os.environ.get("DISPLAY")) is not str or len(os.environ.get("DISPLAY"))==0:
    !bash ../xvfb start
    %env DISPLAY=:1

#
# Import libraries
#
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) # error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay

"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
  mp4list = glob.glob('video/*.mp4')
  if len(mp4list) > 0:
    mp4 = mp4list[0]
    video = io.open(mp4, 'r+b').read()
    encoded = base64.b64encode(video)
    ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
  else: 
    print("Could not find video")
    

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

In [11]:
from spinup import ppo_pytorch as ppo
import torch
import gym
import os.path as osp, time, atexit, os

env_fn = lambda : gym.make('LunarLander-v2')

ac_kwargs = dict(hidden_sizes=[64,64], activation=torch.nn.ReLU)

logger_kwargs = dict(output_dir='gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/%i'%int(time.time()), exp_name='experiment_name')

EPOCH = 20
LOSS = 0.4

In [12]:
ppo(env_fn=env_fn, ac_kwargs=ac_kwargs, steps_per_epoch=500, epochs=20, logger_kwargs=logger_kwargs)  #changed from steps_per_epoch=5000, epochs=250

Logging data to gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/1658605445/progress.txt
Saving config:

{
    "ac_kwargs":	{
        "activation":	"ReLU",
        "hidden_sizes":	[
            64,
            64
        ]
    },
    "actor_critic":	"MLPActorCritic",
    "clip_ratio":	0.2,
    "env_fn":	"<function <lambda> at 0x7fba8833bf80>",
    "epochs":	20,
    "exp_name":	"experiment_name",
    "gamma":	0.99,
    "lam":	0.97,
    "logger":	{
        "<spinup.utils.logx.EpochLogger object at 0x7fba7db548d0>":	{
            "epoch_dict":	{},
            "exp_name":	"experiment_name",
            "first_row":	true,
            "log_current_row":	{},
            "log_headers":	[],
            "output_dir":	"gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/1658605445",
            "output_file":	{
                "<_io.TextIOWrapper name='gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/1658605445/progress.txt' mode='w' encoding=

In [5]:
from spinup.utils.test_policy import load_policy_and_env as load_policy, run_policy
# Show policy
_, get_action = load_policy('gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir')
env2 = gym.make('LunarLander-v2')
env3 = wrap_env(env2)
run_policy(env3, get_action, max_ep_len=500, num_episodes=10)
env3.close()
show_video() 



Loading from gdrive/MyDrive/Reward Side Channels - Model Data/path/to/output_dir/pyt_save/model.pt.


Logging data to /content/gdrive/MyDrive/Reward Side Channels - Model Data/tmp/experiments/1658601484/progress.txt
Episode 0 	 EpRet -87.524 	 EpLen 134
Episode 1 	 EpRet -70.831 	 EpLen 252
Episode 2 	 EpRet -123.811 	 EpLen 321
Episode 3 	 EpRet -12.505 	 EpLen 139
Episode 4 	 EpRet -50.369 	 EpLen 110
Episode 5 	 EpRet -170.749 	 EpLen 341
Episode 6 	 EpRet -45.898 	 EpLen 95
Episode 7 	 EpRet -190.276 	 EpLen 109
Episode 8 	 EpRet -181.170 	 EpLen 152
Episode 9 	 EpRet -66.914 	 EpLen 143
-------------------------------------
|    AverageEpRet |            -100 |
|        StdEpRet |            59.5 |
|        MaxEpRet |           -12.5 |
|        MinEpRet |            -190 |
|           EpLen |             180 |
-------------------------------------
